In [2]:
import glob
import numpy as np
from PIL import *
import PIL.Image

In [3]:
filelist = glob.glob('train/cropped/glu/*.jpg')              #change '/'
glu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(glu.shape)

(40, 497, 497, 3)


In [4]:
filelist = glob.glob('train/cropped/nglu/*.jpg')         #change '/'
nonglu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(nonglu.shape)

(46, 497, 497, 3)


In [5]:
X = np.concatenate((glu,nonglu),axis=0)
print(X.shape)
print(X[1,:].shape)

(86, 497, 497, 3)
(497, 497, 3)


In [6]:
gluL = np.array([1 for i in range(40)])
nongluL = np.array([0 for i in range(46)])
y = np.concatenate((gluL,nongluL),axis=0)
print(y.shape)

(86,)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_test)

(68, 497, 497, 3) (18, 497, 497, 3) (68,) (18,)
[0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0]


In [8]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dropout,Dense,LeakyReLU
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', input_shape=X_train.shape[1:]))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(Conv2D(32, (3,3),padding='valid'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(Conv2D(64, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(Conv2D(128, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(Conv2D(256, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (1,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))

model.add(Flatten())
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(Dense(16))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.10))
model.add(Dense(2, activation='sigmoid'))

/home/subodh_pushkar/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
model.output_shape

(None, 2)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 497, 497, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 497, 497, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 497, 497, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 495, 495, 32)      9248      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 495, 495, 32)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 495, 495, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 247, 247, 32)      0         
__________

# DATA GENERATION

In [11]:
from keras.utils import np_utils
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
num_classes = 2
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                             rotation_range=15, width_shift_range=0.1,
                             height_shift_range=0.1,zoom_range=0.2,
                             fill_mode='constant', cval=0.0)

# print(datagen.fit(X_train))
print(y_train[0])


[1. 0.]


In [12]:
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



# datagen = ImageDataGenerator(
#         horizontal_flip=True, vertical_flip=True,
#                              rotation_range=10, width_shift_range=0.1,
#                              height_shift_range=0.1,zoom_range=0.2,
#                              fill_mode='constant', cval=0.0)

# import glob
# cv_img = []
# for img in glob.glob("train/cropped/glu/*.jpg"):
#     # n= cv2.imread(img)
#     # cv_img.append(n)

#     img = load_img(img)  # this is a PIL image
#     x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
#     x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

#     # the .flow() command below generates batches of randomly transformed images
#     # and saves the results to the `preview/` directory
#     i = 0
#     for batch in datagen.flow(x, batch_size=1,
#                               save_to_dir='train/final_glu_aug', save_prefix='aaaaa', save_format='jpg'):
#         i += 1
#         if i > 10:
#             break  # otherwise the generator would loop indefinitely

In [13]:
# from keras import optimizers
# sgd = optimizers.SGD(lr=0.0003, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.optimizers import *
# EPOCHS = 100
# INIT_LR = 1e-4
# BS = 32
# opt = Adam(lr=INIT_LR)
model.compile(optimizer=SGD(0.0001),loss="binary_crossentropy",metrics=["accuracy"])
# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])
# train the network
print("[INFO] training network...")
H = model.fit_generator(datagen.flow(X_train, y_train, batch_size=1),
                        validation_data=(X_test, y_test), steps_per_epoch=len(X_train), epochs=100, verbose=1)

# model.fit(datagen.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(X_train) / 32, epochs=10,validation_data=(X_test, y_test))

[INFO] training network...
Epoch 1/100
68/68 [==============================] - 84s 1s/step - loss: 0.8803 - acc: 0.5735 - val_loss: 0.6961 - val_acc: 0.4722
Epoch 2/100
68/68 [==============================] - 82s 1s/step - loss: 0.8526 - acc: 0.4412 - val_loss: 0.6928 - val_acc: 0.5000
Epoch 3/100
68/68 [==============================] - 83s 1s/step - loss: 0.6949 - acc: 0.5515 - val_loss: 0.6885 - val_acc: 0.6111
Epoch 4/100
68/68 [==============================] - 83s 1s/step - loss: 0.7256 - acc: 0.4632 - val_loss: 0.6869 - val_acc: 0.6389
Epoch 5/100
68/68 [==============================] - 83s 1s/step - loss: 0.6543 - acc: 0.5882 - val_loss: 0.6825 - val_acc: 0.6111
Epoch 6/100
68/68 [==============================] - 83s 1s/step - loss: 0.7081 - acc: 0.5441 - val_loss: 0.6851 - val_acc: 0.6667
Epoch 7/100
68/68 [==============================] - 83s 1s/step - loss: 0.6680 - acc: 0.6103 - val_loss: 0.6920 - val_acc: 0.5556
Epoch 8/100
68/68 [==============================] - 82s

Epoch 63/100
68/68 [==============================] - 72s 1s/step - loss: 0.4300 - acc: 0.7868 - val_loss: 0.6271 - val_acc: 0.7222
Epoch 64/100
68/68 [==============================] - 80s 1s/step - loss: 0.3925 - acc: 0.8235 - val_loss: 0.6386 - val_acc: 0.6667
Epoch 65/100
68/68 [==============================] - 82s 1s/step - loss: 0.4205 - acc: 0.7941 - val_loss: 0.6324 - val_acc: 0.7222
Epoch 66/100
68/68 [==============================] - 83s 1s/step - loss: 0.3820 - acc: 0.8235 - val_loss: 0.6289 - val_acc: 0.7222
Epoch 67/100
68/68 [==============================] - 82s 1s/step - loss: 0.3696 - acc: 0.8382 - val_loss: 0.6318 - val_acc: 0.7778
Epoch 68/100
68/68 [==============================] - 82s 1s/step - loss: 0.3733 - acc: 0.8309 - val_loss: 0.6243 - val_acc: 0.6944
Epoch 69/100
68/68 [==============================] - 83s 1s/step - loss: 0.4084 - acc: 0.8088 - val_loss: 0.6159 - val_acc: 0.7500
Epoch 70/100
65/68 [===========================>..] - ETA: 3s - loss: 0.4522

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=16)
print(score)

In [ ]:
model.predict(X_test, batch_size=16)
model.predict_classes(X_test,batch_size=16)

In [ ]:
import h5py
model.save_weights('better_weights.h5')